# Scraping Data Rescue Project
github: https://github.com/datarescueproject/portal/

LAST UPDATE: 25/01/2026

In [ ]:
# Imports
import re
import requests
import yaml
import pandas as pd
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from urllib.parse import urlparse
from urllib.parse import unquote
import time


pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)



Getting info from MD files

In [2]:
owner = "datarescueproject"
repo = "portal"
repo_url = f"https://github.com/datarescueproject/portal/"
folder = "_datasets/"
branch = 'main'

In [3]:
ref = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/refs/heads/{branch}").json()
commit_sha = ref['object']['sha']
tree_sha = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/commits/{commit_sha}").json()["tree"]["sha"]
tree = requests.get(f"https://api.github.com/repos/datarescueproject/portal/git/trees/{tree_sha}", params={"recursive":"1"}).json()["tree"]

In [4]:
md_files = []
for item in tree:
    if item["type"] != "blob":
        continue
    if not item["path"].startswith(folder):
        continue
    if not item["path"].lower().endswith(".md"):
        continue
    md_files.append(item["path"])

md_files = sorted(md_files)
print(len(md_files))

1978


In [6]:
frontmatter_regex = re.compile(
    r'^\s*---\s*\n(.*?)\n---\s*\n?',
    re.DOTALL
)

projects = []
resources = []

for path in md_files:
    raw_url = f'https://raw.githubusercontent.com/datarescueproject/portal/main/{path}'
    markdown = requests.get(raw_url).text

    match = frontmatter_regex.match(markdown)
    if not match:
        frontmatter = {}
    else:
        frontmatter = yaml.safe_load(match.group(1)) or {}

    if not isinstance(frontmatter, dict):
        frontmatter = {}

    project_row = {
        'file': path,
        'schema': frontmatter.get('schema'),
        'title': frontmatter.get('title'),
        'organization': frontmatter.get('organization'),
        'agency': frontmatter.get('agency'),
        'websites': frontmatter.get('websites'),
        'data_source': frontmatter.get('data_source'),
        'description': frontmatter.get('description'),
        'last_modified': frontmatter.get('last_modified'),
        'metadata_available': frontmatter.get('metadata_available'),
        'metadata_url': frontmatter.get('metadata_url'),
    }

    categories = frontmatter.get('category', [])
    if isinstance(categories, list):
        project_row['category'] = '; '.join(map(str, categories))
    elif categories:
        project_row['category'] = str(categories)
    else:
        project_row['category'] = ''

    resource_list = frontmatter.get('resources', [])
    if not isinstance(resource_list, list):
        resource_list = []

    project_row['resource_count'] = len(resource_list)
    projects.append(project_row)

    for res in resource_list:
        if not isinstance(res, dict):
            continue

        resources.append({
            'file': path,
            'project_title': frontmatter.get('title'),
            'id': res.get('id'),
            'url': res.get('url'),
            'format': res.get('format'),
            'status': res.get('status'),
            'size': res.get('size'),
            'download_date': res.get('download_date'),
            'maintainer': res.get('maintainer'),
            'notes': res.get('notes'),
        })

projects_df = pd.DataFrame(projects)
resources_df = pd.DataFrame(resources)

projects_df.head()

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-charges/10j-injunctions,None,2025-04-20,True,None,Labor & Employment,1
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,Climate & Environment,1
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Operations and Services (ITOPS),Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1
4,_datasets/2016-americorps-mes-americorps-member-exit-survey.md,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2016-AmeriCorps-MES-AmeriCorps-Member-Exit-Survey/wqhv-fm5d/about_data,None,2025-03-10,False,None,Humanitarian & Disaster Relief; Military & Veterans Affairs,1


In [7]:
projects_df.to_csv('datarescuetracker.csv')
resources_df.to_csv('resources.csv')

In [8]:
projects_df

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-charges/10j-injunctions,None,2025-04-20,True,None,Labor & Employment,1
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,Climate & Environment,1
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Operations and Services (ITOPS),Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1
4,_datasets/2016-americorps-mes-americorps-member-exit-survey.md,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2016-AmeriCorps-MES-AmeriCorps-Member-Exit-Survey/wqhv-fm5d/about_data,None,2025-03-10,False,None,Humanitarian & Disaster Relief; Military & Veterans Affairs,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1973,_datasets/yemen-usaid-education-data.md,data_rescue_project,Yemen USAID Education Data,M/CIO Development Information Solution (DIS),U.S. Agency for International Development,data.usaid.gov,data.usaid.gov,None,2025-11-03,False,None,Education; Humanitarian & Disaster Relief,1
1974,_datasets/your-right-to-federal-records.md,data_rescue_project,Your Right to Federal Records,Federal Mediation and Conciliation Service,Federal Mediation and Conciliation Service,fmcs.gov,https://www.fmcs.gov/resources/documents-and-data/,None,2025-03-31,False,None,Labor & Employment,1
1975,_datasets/youth-risk-behavior-surveillance-system-yrbss-2023-2025.md,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS) 2023-2025,Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/yrbs/about/index.html,None,2025-11-05,False,None,Health & Healthcare,1
1976,_datasets/youth-risk-behavior-surveillance-system-yrbss.md,data_rescue_project,Youth Risk Behavior Surveillance System (YRBSS),Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,cdc.gov,https://www.cdc.gov/index.html,None,2025-05-19,True,None,Health & Healthcare,2


In [11]:
projects_df.shape

(1978, 13)

In [9]:
resources_df

,file,project_title,id,url,format,status,size,download_date,maintainer,notes
0,_datasets/10j-injunctions.md,10(j) Injunctions,759,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL","Section 10(j) of the National Labor Relations Act authorizes the National Labor Relations Board to seek temporary injunctions against employers and unions in federal district courts to stop unfair labor practices while the case is being litigated before administrative law judges and the Board. These temporary injunctions are needed to protect the process of collective bargaining and employee rights under the Act, and to ensure that Board decisions will be meaningful. The section was added as part of a set of reforms to the Act in 1947. Over the years, all NLRB General Counsels have made use of this effective enforcement tool, as shown in this chart.The csv contains Authorization Dates, Case Numbers, Case Names, and Injunction Status as of the date collected (2025-04-07). This list is all 10(j) injunction cases authorized by the Board since September 1, 2010."
1,_datasets/2002-2024-mbda-grantees.md,2002-2024 MBDA Grantees,696,https://www.datalumos.org/datalumos/project/223443/version/V1/view,CSV,Finished,0.000,2025-03-18,"DRP, DL",None
2,_datasets/2006-iur-public-database.md,2006 IUR Public Database,1280,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC","There was a 2006 version and a 2002-1986 version. Both are archived, hence 2 dataverse URLs. ~ag. Seperate Metadata https://www.epa.gov/chemical-data-reporting/summary-cdr-reporting-requirements-year, https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database"
3,_datasets/2014-minority-veteran-report.md,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/222881/version/V1/view,CSV,Finished,0.000,2025-03-11,"DRP, DL",None
4,_datasets/2016-americorps-mes-americorps-member-exit-survey.md,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/222043/version/V1/view,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None
...,...,...,...,...,...,...,...,...,...,...
2021,_datasets/your-right-to-federal-records.md,Your Right to Federal Records,643,https://www.datalumos.org/datalumos/project/223063/version/V1/view,PDF,Finished,0.000,2025-03-15,"DRP, DL",Regulations and Policies Tab
2022,_datasets/youth-risk-behavior-surveillance-system-yrbss-2023-2025.md,Youth Risk Behavior Surveillance System (YRBSS) 2023-2025,1502,https://www.datalumos.org/datalumos/project/220761/view,None,Finished,NaN,None,DL,None
2023,_datasets/youth-risk-behavior-surveillance-system-yrbss.md,Youth Risk Behavior Surveillance System (YRBSS),326,https://www.dropbox.com/scl/fo/2t0ehrteq62jqrj0slm3q/AHed9WK-9ydPJT1398w42zo?rlkey=pqb2fisu1rgjjc2badfyfq0nk&dl=0,"DTA, XLSX, do, MDB, dat, SPS, SAS",Finished,0.000,2025-01-31,ICPSR,None
2024,_datasets/youth-risk-behavior-surveillance-system-yrbss.md,Youth Risk Behavior Surveillance System (YRBSS),1040,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/31/health-policy/yrbss/data.zip,ZIP,Finished,0.877,2025-02-05,UI,"The Youth Risk Behavior Surveillance System focuses on health risk behaviors that are often established during childhood and early adolescence, including behaviors associated with tobacco use, alcohol and other drug use, unintentional injuries, sexual behaviors related to unintended pregnancy and sexually transmitted infections, unhealthy diet, and inadequate physical activity."


In [12]:
df = pd.merge(projects_df, resources_df, on='file')

In [13]:
df

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-charges/10j-injunctions,None,2025-04-20,True,None,Labor & Employment,1,10(j) Injunctions,759,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL","Section 10(j) of the National Labor Relations Act authorizes the National Labor Relations Board to seek temporary injunctions against employers and unions in federal district courts to stop unfair labor practices while the case is being litigated before administrative law judges and the Board. These temporary injunctions are needed to protect the process of collective bargaining and employee rights under the Act, and to ensure that Board decisions will be meaningful. The section was added as part of a set of reforms to the Act in 1947. Over the years, all NLRB General Counsels have made use of this effective enforcement tool, as shown in this chart.The csv contains Authorization Dates, Case Numbers, Case Names, and Injunction Status as of the date collected (2025-04-07). This list is all 10(j) injunction cases authorized by the Board since September 1, 2010."
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1,2002-2024 MBDA Grantees,696,https://www.datalumos.org/datalumos/project/223443/version/V1/view,CSV,Finished,0.000,2025-03-18,"DRP, DL",None
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,Climate & Environment,1,2006 IUR Public Database,1280,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC","There was a 2006 version and a 2002-1986 version. Both are archived, hence 2 dataverse URLs. ~ag. Seperate Metadata https://www.epa.gov/chemical-data-reporting/summary-cdr-reporting-requirements-year, https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database"
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Operations and Services (ITOPS),Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/222881/version/V1/view,CSV,Finished,0.000,2025-03-11,"DRP, DL",None
4,_datasets/2016-americorps-mes-americorps-member-exit-survey.md,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2016-AmeriCorps-MES-AmeriCorps-Member-Exit-Survey/wqhv-fm5d/about_data,None,2025-03-10,False,None,Humanitarian & Disaster Relief; Military & Veterans Affairs,1,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/222043/version/V1/view,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,_datasets/your-right-to-federal-records.md,data_rescue_project,Your Right to Federal Records,Federal Mediation and Conciliation Service,Federal Mediation and Conciliation Service,fmcs.gov,https://www.fmcs.gov/resou

extracting domains

In [14]:
def get_domain(url):
    if not isinstance(url, str):
        return None
    return urlparse(url).netloc

In [15]:
df["domain"] = df["url"].apply(get_domain)
df.head()

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain
0,_datasets/10j-injunctions.md,data_rescue_project,10(j) Injunctions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/what-we-do/investigate-charges/10j-injunctions,None,2025-04-20,True,None,Labor & Employment,1,10(j) Injunctions,759,https://doi.org/10.3886/E226824V1,"CSV, TXT",Finished,0.000,2025-04-07,"DRP, DL","Section 10(j) of the National Labor Relations Act authorizes the National Labor Relations Board to seek temporary injunctions against employers and unions in federal district courts to stop unfair labor practices while the case is being litigated before administrative law judges and the Board. These temporary injunctions are needed to protect the process of collective bargaining and employee rights under the Act, and to ensure that Board decisions will be meaningful. The section was added as part of a set of reforms to the Act in 1947. Over the years, all NLRB General Counsels have made use of this effective enforcement tool, as shown in this chart.The csv contains Authorization Dates, Case Numbers, Case Names, and Injunction Status as of the date collected (2025-04-07). This list is all 10(j) injunction cases authorized by the Board since September 1, 2010.",doi.org
1,_datasets/2002-2024-mbda-grantees.md,data_rescue_project,2002-2024 MBDA Grantees,Minority Business Development Agency,Department of Commerce,mbda.gov,https://www.mbda.gov/research/data/mbda-grantees,None,2025-04-02,False,None,Business & Economy,1,2002-2024 MBDA Grantees,696,https://www.datalumos.org/datalumos/project/223443/version/V1/view,CSV,Finished,0.000,2025-03-18,"DRP, DL",None,www.datalumos.org
2,_datasets/2006-iur-public-database.md,data_rescue_project,2006 IUR Public Database,Environmental Protection Agency,Environmental Protection Agency,epa.gov,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,None,2025-09-18,True,https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database,Climate & Environment,1,2006 IUR Public Database,1280,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,"mhtml, ZIP, PDF",Finished,0.005,2025-02-26,"HD, CAFE-RCC","There was a 2006 version and a 2002-1986 version. Both are archived, hence 2 dataverse URLs. ~ag. Seperate Metadata https://www.epa.gov/chemical-data-reporting/summary-cdr-reporting-requirements-year, https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database",dataverse.harvard.edu
3,_datasets/2014-minority-veteran-report.md,data_rescue_project,2014 Minority Veteran Report,Office of Information and Technology - IT Operations and Services (ITOPS),Department of Veterans Affairs,data.va.gov,https://www.data.va.gov/stories/s/gavm-n6bm,None,2025-03-17,False,None,Science & Research; Military & Veterans Affairs,1,2014 Minority Veteran Report,412,https://www.datalumos.org/datalumos/project/222881/version/V1/view,CSV,Finished,0.000,2025-03-11,"DRP, DL",None,www.datalumos.org
4,_datasets/2016-americorps-mes-americorps-member-exit-survey.md,data_rescue_project,2016 AmeriCorps MES AmeriCorps Member Exit Survey,AmeriCorps,AmeriCorps,data.americorps.gov,https://data.americorps.gov/National-Service/2016-AmeriCorps-MES-AmeriCorps-Member-Exit-Survey/wqhv-fm5d/about_data,None,2025-03-10,False,None,Humanitarian & Disaster Relief; Military & Veterans Affairs,1,2016 AmeriCorps MES AmeriCorps Member Exit Survey,396,https://www.datalumos.org/datalumos/project/222043/version/V1/view,"CSV, XLSX, PDF",Finished,0.000,2025-03-08,"DRP, DL",None,www.datalumos.org


In [16]:
df['domain'].value_counts()

domain
www.datalumos.org                                1589
www.dropbox.com                                    85
sciop.net                                          72
doi.org                                            49
dataverse.harvard.edu                              46
zenodo.org                                         34
archive.org                                        28
github.com                                         22
urban-data-catalog.s3.us-east-1.amazonaws.com      10
purl.stanford.edu                                  10
nlrbresearch.com                                    4
web.archive.org                                     3
arcgis.com                                          3
www.openicpsr.org                                   3
www.arcgis.com                                      3
biglocalnews.org                                    3
doi.pangaea.de                                      2
www.hydroshare.org                                  1
s3.amazonaws.com     

In [17]:
df[df['domain'].isna()]

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain
134,_datasets/bipartisan-infrastructure-law-and-inflation-reduction-act-awards-explorer.md,data_rescue_project,Bipartisan Infrastructure Law and Inflation Reduction Act Awards Explorer,National Oceanic and Atmospheric Administration,Department of Commerce,noaa.gov,https://www.noaa.gov/bil-ira-awards-explorer,None,2025-03-02,False,None,Climate & Environment,1,Bipartisan Infrastructure Law and Inflation Reduction Act Awards Explorer,211,None,CSV,Finished,0.000,None,"EDGI, ESRI",Local - EDGI; only downloaded the csv; not trying to recreate the mapper,None
135,_datasets/bls-downloads.md,data_rescue_project,BLS Downloads,Bureau of Labor Statistics,Department of Labor,download.bls.gov,https://download.bls.gov,None,2025-02-10,False,None,Labor & Employment; Business & Economy,1,BLS Downloads,1,None,None,In Progress,47.000,2025-02-01,DRP,None,None
143,_datasets/building-code-adoption-tracking.md,data_rescue_project,Building Code Adoption Tracking,Federal Emergency Management Agency,Department of Homeland Security,fema.gov,https://stantec.maps.arcgis.com/apps/MapSeries/index.html?appid=a053ac48343c4217ab4184bc8759c350,None,2025-03-03,False,None,Climate & Environment; Humanitarian & Disaster Relief,1,Building Code Adoption Tracking,205,None,"Interactive Map, PDF",In Progress,0.000,None,ESRI,"Local - ESRI; Identify which building codes an area has adopted by hazard risk, PDF fact sheets by region",None
151,_datasets/cdc-ftp-server.md,data_rescue_project,CDC FTP Server,Centers for Disease Control and Prevention (CDC),Department of Health and Human Services,ftp.cdc.gov,https://ftp.cdc.gov/,None,2025-05-23,False,None,Health & Healthcare,1,CDC FTP Server,2,None,None,In Progress,213.000,2025-02-01,DRP,None,None
160,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,None,Housing & Community Development; Health & Healthcare; Labor & Employment; Business & Economy; Social Services; Education,4,Census Bureau FTP Server,3,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,180.000,2025-02-01,DRP,Partial download.,None
161,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,None,Housing & Community Development; Health & Healthcare; Labor & Employment; Business & Economy; Social Services; Education,4,Census Bureau FTP Server,1050,None,"PDF, ZIP, XLS, HTML, TXT",Uploading,6200.000,2025-02-17,"DRP, ANON","Created as a torrent by Reddit user enchanting_endeavour and can be downloaded using the following magnet link magnet?xt=urnbtihda7f54c14ca6ab795ddb9f87b953c3dd8f22fbcd&dn=ftp2_census_gov_2025_02_17_torrents&tr=http%3A%2F%2Fwww.torrentsnipe.info%3A2701%2Fannounce&tr=udp%3A%2F%2Fdiscord.heihachi.pw%3A6969%2Fannounce. Original description ""In order to make the data manageable, the whole dataset is broken up into 41 pieces. This file is a torrent of torrents which has a torrent file for each of those pieces. Note that due to an error on my part, piece 31 is just an empty data structure and has no other data in it. The top level directory, ftp2.census.gov, in each of the pieces should merge cleanly without conflicts with the others. This dataset includes data for many of the census.gov subdomains, though I have not be able to verify which domains specifically are included and which are not.""",None
162,_datasets/census-bureau-ftp-server.md,data_rescue_project,Census Bureau FTP Server,Census Bureau,Department of Commerce,www2.census.gov,https://www2.census.gov,None,2025-05-23,False,None,Housing & Community Development; Health & Healthcare; Labor & Employment; Business & Economy; Social Service

Scraping datalumos for metadata (didnt work)

In [ ]:
!pip -q install playwright beautifulsoup4 lxml
!playwright install chromium

In [ ]:
doi_re = re.compile(r"(?:doi\.org/|doi:\s*)(10\.\d{4,9}/[-._;()/:A-Z0-9]+)", re.I)

mask = resources_df["domain"].isin(["www.datalumos.org", "datalumos.org"])
cols = ["principal_investigators","summary","project_title_page","geographic_coverage","time_periods","doi"]
resources_df.loc[mask, cols] = None

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()

    for i, row in resources_df[mask].iterrows():
        url = row["url"]
        print(url)
        try:
            await page.goto(url, wait_until="networkidle", timeout=60000)
            html = await page.content()
            soup = BeautifulSoup(html, "lxml")

            # scan all <strong> labels and capture the text in the same parent
            for strong in soup.find_all("strong"):
                label = strong.get_text(" ", strip=True)
                parent_text = strong.parent.get_text(" ", strip=True)
                value = re.sub(rf"^{re.escape(label)}\s*:?\s*", "", parent_text).strip()

                low = label.lower()
                if "principal investigator" in low:
                    resources_df.at[i, "principal_investigators"] = value
                elif low.startswith("summary"):
                    resources_df.at[i, "summary"] = value
                elif low.startswith("project title"):
                    resources_df.at[i, "project_title_page"] = value
                elif low.startswith("geographic coverage"):
                    resources_df.at[i, "geographic_coverage"] = value
                elif low.startswith("time period"):
                    resources_df.at[i, "time_periods"] = value

            text = soup.get_text(" ", strip=True)
            found = doi_re.findall(text)
            if found:
                resources_df.at[i, "doi"] = found[0]

        except Exception as e:
            print("Failed:", url, "|", e)

    await browser.close()

resources_df.loc[mask, ["url"] + cols].head()

In [ ]:
from playwright.async_api import async_playwright

test_url = resources_df.loc[mask, "url"].dropna().iloc[0]
print("Testing:", test_url)

async with async_playwright() as p:
    browser = await p.chromium.launch(headless=True)
    page = await browser.new_page()
    await page.goto(test_url, timeout=60000)
    print(await page.title())
    await browser.close()

Filling in doi column for doi we already have

In [18]:
dois_provided = df['domain'] == 'doi.org'

In [80]:
doi_pattern = r"(10\.\d{4,9}/[-._;()/:a-z0-9]+)"

df.loc[dois_provided, "doi"] = (
    df.loc[dois_provided, "url"]
      .str.lower()
      .str.extract(doi_pattern, expand=False)
)

In [81]:
df['doi'].isna().mean()

0.15301085883514315

Trying to get doi from urls for datalumos projects based n project number in the url

In [21]:
m = df["url"].astype(str).str.extract(r"/project/(\d+)/version/(V\d+)/", expand=True)
df["project_id"] = m[0]
df["version"] = m[1]

# guess DOI in the common ICPSR style: 10.3886/E{project_id}{version}
df["doi_guess"] = None
mask_dl = df["domain"].isin(["www.datalumos.org", "datalumos.org"]) & df["project_id"].notna() & df["version"].notna()
df.loc[mask_dl, "doi_guess"] = "10.3886/E" + df.loc[mask_dl, "project_id"] + df.loc[mask_dl, "version"]

df.loc[mask_dl, ["url","project_id","version","doi_guess"]]

,url,project_id,version,doi_guess
1,https://www.datalumos.org/datalumos/project/223443/version/V1/view,223443,V1,10.3886/E223443V1
3,https://www.datalumos.org/datalumos/project/222881/version/V1/view,222881,V1,10.3886/E222881V1
4,https://www.datalumos.org/datalumos/project/222043/version/V1/view,222043,V1,10.3886/E222043V1
5,https://www.datalumos.org/datalumos/project/222044/version/V2/view,222044,V2,10.3886/E222044V2
6,https://www.datalumos.org/datalumos/project/222044/version/V2/view,222044,V2,10.3886/E222044V2
...,...,...,...,...
2010,https://www.datalumos.org/datalumos/project/223962/version/V1/view,223962,V1,10.3886/E223962V1
2013,https://www.datalumos.org/datalumos/project/222882/version/V1/view,222882,V1,10.3886/E222882V1
2014,https://www.datalumos.org/datalumos/project/227293/version/V1/view,227293,V1,10.3886/E227293V1
2015,https://www.datalumos.org/datalumos/project/227006/version/V1/view,227006,V1,10.3886/E227006V1


In [22]:
df['doi_guess'].value_counts()

doi_guess
10.3886/E222581V1    51
10.3886/E223141V1    48
10.3886/E229201V1    37
10.3886/E223001V1    30
10.3886/E224621V1    25
                     ..
10.3886/E240512V1     1
10.3886/E240514V1     1
10.3886/E240516V1     1
10.3886/E240543V1     1
10.3886/E227293V1     1
Name: count, Length: 969, dtype: int64

testing if it works

In [23]:
test = "10.3886/E223443V1"

url = f"https://api.datacite.org/dois/{test}"

headers = {
    "Accept": "application/vnd.api+json"
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":{"id":"10.3886/e223443v1","type":"dois","attributes":{"doi":"10.3886/e223443v1","prefix":"10.3886","suffix":"e223443v1","identifiers":[],"alternateIdentifiers":[],"creators":[{"name":"United States Department Of Commerce. Minority Business Development Agency","nameType":"Organizational","affiliation":[],"nameIdentifiers":[]}],"titles":[{"lang":"en","title":"2022-2024 MBDA Grantees"}],"publisher":"ICPSR - Interuniversity Consortium for Political and Social Research","container":{},"publicationYear":2025,"subjects":[{"lang":"en","subject":"grants"},{"lang":"en","subject":"minority businesses"}],"contributors":[],"dates":[{"date":"2022-01-01/2024-12-31","dateType":"Collected"},{"date":"2025","dateType":"Issued"}],"language":"en","types":{"ris":"DATA","bibtex":"misc","citeproc":"dataset","schemaOrg":"Dataset","resourceTypeGeneral":"Dataset"},"relatedIdentifiers":[{"relationType":"IsVersionOf","relatedIdentifier":"10.3886/e223443","relatedIdentifierType":"DOI"}],"relatedItems":[],"s

dois are correct, so fill in doi col with guesses

In [24]:
df["doi"] = df["doi"].combine_first(df["doi_guess"])

get dois of data outside datalumos

In [82]:
missing_doi = df[df["doi"].isna()]

In [83]:
missing_doi

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess
44,_datasets/administrative-law-judge-decisions.md,data_rescue_project,Administrative Law Judge Decisions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/cases-decisions/decisions/administrative-law-judge-decisions,None,2025-05-14,False,None,Labor & Employment,1,Administrative Law Judge Decisions,1008,https://nlrbresearch.com/NLRB/NLRB_DB?_search=type%3A+%22ALJ%22,PDF,Finished,0.000,None,NLRB-R,Captured as part of NLRB Research a free database a researcher made.,nlrbresearch.com,None,NaN,NaN,None
61,_datasets/affirmatively-furthering-fair-housing-affh-data.md,data_rescue_project,Affirmatively Furthering Fair Housing (AFFH) Data,Department of Housing and Urban Development,Department of Housing and Urban Development,hud.gov,https://www.hud.gov/AFFH,None,2025-05-19,True,https://datacatalog.urban.org/dataset/us-department-housing-and-urban-development-affirmatively-furthering-fair-housing-hud-affh,Housing & Community Development,1,Affirmatively Furthering Fair Housing (AFFH) Data,1037,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/04/03/housing-and-communities/hud-affh/data.zip,ZIP,Finished,0.876,2024-12-18,UI,"This dataset contains all data, documentation, and file resources linked to on the main US Department of Housing and Urban Development’s Affirmatively Furthering Fair Housing (AFFH) page and powering the AFFH tool.",urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
65,_datasets/aging-independence-and-disability-agid-program-portal-data.md,data_rescue_project,"AGing, Independence, and Disability (AGID) Program Portal Data",Administration for Community Living,Department of Health and Human Services,agid.acl.gov,https://agid.acl.gov/release.html,None,2025-05-19,True,https://datacatalog.urban.org/dataset/aging-independence-and-disability-agid-program-portal-data,Social Services; Health & Healthcare,1,"AGing, Independence, and Disability (AGID) Program Portal Data",1041,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/28/race-and-equity/agid/data.zip,ZIP,Finished,6.000,2025-03-28,UI,"This resource contains all data from AGing, Independence, and Disability’s Program Data Portal from the US Department of Health and Human Services’ Administration for Community Living.",urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
67,_datasets/aids-drug-assistance-programs-adaps.md,data_rescue_project,AIDS Drug Assistance Programs (ADAPS),Health Resources and Services Administration,Department of Health and Human Services,hrsa.gov,https://www.hrsa.gov/grants/find-funding/HRSA-23-056,None,2025-03-26,False,None,Health & Healthcare,1,AIDS Drug Assistance Programs (ADAPS),242,https://www.dropbox.com/scl/fo/60drbfxp3p1hb1l4gr3cj/AHrpM10MrLH-0MvQHCcXFI4?rlkey=fps9vit5x2cg6367jhdykkl9a&dl=0,CSV,Finished,0.000,2025-01-31,ICPSR,None,www.dropbox.com,None,NaN,NaN,None
68,_datasets/air-data-pre-generated-files-dec-2024.md,data_rescue_project,"Air Data Pre-Generated Files, Dec. 2024",Environmental Protection Agency,Environmental Protection Agency,aqs.epa.gov,https://aqs.epa.gov/aqsweb/airdata/download_files.html,None,2025-05-18,False,None,Climate & Environment,1,"Air Data Pre-Generated Files, Dec. 2024",1029,https://sciop.net/uploads/75bd916972ae78cbe59534dd88da55d11c4719f2,CSV,Finished,20.000,2025-05-15,"DRP, SRC",Alternate torrent location https://academictorrents.com/details/75bd916972ae78cbe59534dd88da55d11c4719f2,sciop.net,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017,_datasets/world-ocean-atlas-2023-figures.md,data_rescue_project,World Ocean Atlas 2023 Figures,National Oceanic and Atmosp

In [84]:
missing_doi['domain'].value_counts()

domain
www.dropbox.com                                  85
sciop.net                                        72
archive.org                                      28
github.com                                       22
purl.stanford.edu                                10
urban-data-catalog.s3.us-east-1.amazonaws.com    10
nlrbresearch.com                                  4
web.archive.org                                   3
arcgis.com                                        3
www.openicpsr.org                                 3
www.arcgis.com                                    3
biglocalnews.org                                  3
www.datalumos.org                                 2
doi.pangaea.de                                    2
s3.amazonaws.com                                  1
tables.codeberg.page                              1
www.thelgbtqarchive.org                           1
www.hydroshare.org                                1
box.hu-berlin.de                                  1
erica

In [85]:
missing_doi['domain'].unique()

array(['nlrbresearch.com',
       'urban-data-catalog.s3.us-east-1.amazonaws.com', 'www.dropbox.com',
       'sciop.net', 'livingatlas.arcgis.com', 'web.archive.org',
       'archive.org', None, 'github.com', 'www.datalumos.org',
       'purl.stanford.edu', 'source.coop', 'hub.arcgis.com',
       'doi.pangaea.de', 'www.zelma.ai', 'erica.datarescueproject.org',
       'edgi-govdata-archiving.github.io', 'public.tableau.com',
       'biglocalnews.org', 'gofile.me', 'www.documentcloud.org',
       's3.amazonaws.com', 'box.hu-berlin.de', 'www.arcgis.com',
       'www.openicpsr.org', 'arcgis.com', 'www.thelgbtqarchive.org',
       'tables.codeberg.page', 'www.hydroshare.org', 'data.openei.org'],
      dtype=object)

In [90]:
missing_doi[missing_doi['domain'] == 'urban-data-catalog.s3.us-east-1.amazonaws.com']

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess
61,_datasets/affirmatively-furthering-fair-housing-affh-data.md,data_rescue_project,Affirmatively Furthering Fair Housing (AFFH) Data,Department of Housing and Urban Development,Department of Housing and Urban Development,hud.gov,https://www.hud.gov/AFFH,None,2025-05-19,True,https://datacatalog.urban.org/dataset/us-department-housing-and-urban-development-affirmatively-furthering-fair-housing-hud-affh,Housing & Community Development,1,Affirmatively Furthering Fair Housing (AFFH) Data,1037,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/04/03/housing-and-communities/hud-affh/data.zip,ZIP,Finished,0.876,2024-12-18,UI,"This dataset contains all data, documentation, and file resources linked to on the main US Department of Housing and Urban Development’s Affirmatively Furthering Fair Housing (AFFH) page and powering the AFFH tool.",urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
65,_datasets/aging-independence-and-disability-agid-program-portal-data.md,data_rescue_project,"AGing, Independence, and Disability (AGID) Program Portal Data",Administration for Community Living,Department of Health and Human Services,agid.acl.gov,https://agid.acl.gov/release.html,None,2025-05-19,True,https://datacatalog.urban.org/dataset/aging-independence-and-disability-agid-program-portal-data,Social Services; Health & Healthcare,1,"AGing, Independence, and Disability (AGID) Program Portal Data",1041,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/28/race-and-equity/agid/data.zip,ZIP,Finished,6.000,2025-03-28,UI,"This resource contains all data from AGing, Independence, and Disability’s Program Data Portal from the US Department of Health and Human Services’ Administration for Community Living.",urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
109,_datasets/associate-attorney-general-select-publications.md,data_rescue_project,Associate Attorney General Select Publications,Justice Management Division,Department of Justice,justice.gov,https://www.justice.gov/asg/select-publications,None,2025-05-19,True,https://datacatalog.urban.org/dataset/us-department-justice-associate-attorney-general-select-publications,Justice & Public Safety,1,Associate Attorney General Select Publications,1038,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/28/justice-and-safety/doj-associate-attorney-general-publications/data.zip,ZIP,Finished,0.015,2025-01-15,UI,This resource contains select publications from the US Department of Justice’s associate attorney general.,urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
114,_datasets/attorney-general-publications.md,data_rescue_project,Attorney General Publications,Justice Management Division,Department of Justice,justice.gov,https://www.justice.gov/ag/select-publications,None,2025-05-19,True,https://datacatalog.urban.org/dataset/us-department-justice-attorney-general-publications,Justice & Public Safety,1,Attorney General Publications,1043,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/28/justice-and-safety/doj-attorney-general-publications/data.zip,ZIP,Finished,0.034,2024-12-27,UI,This resource contains select publications from the attorney general during the Biden administration.,urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None
186,_datasets/climate-and-economic-justice-screening-tool-cejst.md,data_rescue_project,Climate and Economic Justice Screening Tool (CEJST),Environmental Protection Agency,Environmental Protection Agency,ejscreen.epa.gov,https://www.epa.gov/ejscreen,None,2025-09-02,True,https://datacatalog.urban.org/dataset/climate-and-economic-justice-screening-tool-data,Climate & Environment,2,Cl

dataverse dois

In [68]:
missing_doi[missing_doi['domain'] == 'dataverse.harvard.edu']

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess


In [56]:
is_harvard = (df['domain'].eq('dataverse.harvard.edu') & df['url'].notna())

df.loc[is_harvard, 'doi_guess'] = (
    df.loc[is_harvard, 'url']
      .astype(str)
      .str.extract(r'persistentId=([^&#]+)', expand=False)
      .apply(lambda x: unquote(x) if pd.notna(x) else pd.NA)
      .str.lower()
      .str.strip()
      .str.replace('doi:', '', regex=False)
      .str.replace('doi', '', regex=False)
)

df.loc[is_harvard, ['url', 'doi_guess']].head()

,url,doi_guess
2,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/F3A62W,10.7910/dvn/f3a62w
107,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/MWNGDP,10.7910/dvn/mwngdp
110,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/U7WVNO,10.7910/dvn/u7wvno
129,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi%3A10.7910%2FDVN%2FFRAGKR,10.7910/dvn/fragkr
133,https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi10.7910/DVN/0L9K3E,10.7910/dvn/0l9k3e


In [57]:
test = "10.7910/dvn/f3a62w"

url = f"https://api.datacite.org/dois/{test}"

headers = {
    "Accept": "application/vnd.api+json"
}

response = requests.get(url, headers=headers)

print(response.text)

{"data":{"id":"10.7910/dvn/f3a62w","type":"dois","attributes":{"doi":"10.7910/dvn/f3a62w","prefix":"10.7910","suffix":"dvn/f3a62w","identifiers":[],"alternateIdentifiers":[],"creators":[{"name":"EPA","nameType":"Organizational","affiliation":["U.S. EPA"],"nameIdentifiers":[]}],"titles":[{"title":"Extracted Data From: Downloadable 2006 IUR Public Database"}],"publisher":"Harvard Dataverse","container":{},"publicationYear":2025,"subjects":[{"subject":"Chemistry"},{"subject":"Earth and Environmental Sciences"},{"subject":"Environmental Health","schemeUri":"https://tools.niehs.nih.gov/cchhglossary/"},{"subject":"Exposure"}],"contributors":[{"name":"CAFE","nameType":"Personal","affiliation":[],"contributorType":"ContactPerson","nameIdentifiers":[]}],"dates":[{"date":"2025-02-18","dateType":"Submitted"},{"date":"2025-02-26","dateType":"Available"},{"date":"2025-02-26","dateType":"Updated"},{"date":"2006-01-01/2006-12-31","dateType":"Other","dateInformation":"Time period covered by the data"}

In [58]:
df["doi"] = df["doi"].fillna(df["doi_guess"])


zenodo dois

In [70]:
missing_doi[missing_doi['domain'] == 'zenodo.org'].head()

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess
92,_datasets/annual-energy-outlook.md,data_rescue_project,Annual Energy Outlook,US Department of Energy - Office of the CIO,Department of Energy,eia.gov,https://www.eia.gov/outlooks/aeo/data/browser/,None,2025-02-12,False,None,Energy,1,Annual Energy Outlook,73,https://zenodo.org/records/10838488,"XLSX, ZIP",Finished,6.7,2025-03-19,"PEDP, CaCo",None,zenodo.org,None,NaN,NaN,None
96,_datasets/annual-technology-baseline-atb-for-electricity-and-transportation.md,data_rescue_project,Annual Technology Baseline (ATB) for Electricity and Transportation,National Renewable Energy Laboratory,Department of Energy,nrel.gov,https://atb.nrel.gov/,None,2025-03-23,False,None,Climate & Environment; Energy,1,Annual Technology Baseline (ATB) for Electricity and Transportation,234,https://zenodo.org/records/14784563,"Parquet, XLSX, JSON",Finished,3.7,2024-08-01,"PEDP, CaCo",None,zenodo.org,None,NaN,NaN,None
182,_datasets/clean-air-markets-program-data-campd-daily-emissions-for-2020-by-state.md,data_rescue_project,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2020 by State,Environmental Protection Agency,Environmental Protection Agency,campd.epa.gov,https://campd.epa.gov/data,None,2025-10-01,False,None,Climate & Environment,1,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2020 by State,1300,https://zenodo.org/records/15047648,CSV,Finished,5.9,2025-03-18,"PEDP, EDGI",None,zenodo.org,None,NaN,NaN,None
183,_datasets/clean-air-markets-program-data-campd-daily-emissions-for-2024-by-quarter.md,data_rescue_project,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2024 by Quarter,Environmental Protection Agency,Environmental Protection Agency,campd.epa.gov,https://campd.epa.gov/data,None,2025-10-01,False,None,Climate & Environment,1,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2024 by Quarter,1299,https://zenodo.org/records/15047486,CSV,Finished,6.5,2025-03-18,"PEDP, EDGI",None,zenodo.org,None,NaN,NaN,None
184,_datasets/clean-air-markets-program-data-campd-daily-emissions-for-2024-by-state.md,data_rescue_project,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2024 by State,Environmental Protection Agency,Environmental Protection Agency,campd.epa.gov,https://campd.epa.gov/data,None,2025-10-01,False,None,Climate & Environment,1,Clean Air Markets Program Data (CAMPD) Daily Emissions for 2024 by State,1302,https://zenodo.org/records/15047671,CSV,Finished,3.0,2025-03-18,"PEDP, EDGI",None,zenodo.org,None,NaN,NaN,None


In [71]:
is_zenodo = (df['domain'].eq('zenodo.org') & df['url'].notna())

df.loc[is_zenodo, 'doi_guess'] = (
    df.loc[is_zenodo, 'url']
      .astype(str)
      .str.extract(r'/records?/(\d+)', expand=False)
      .apply(lambda x: f'10.5281/zenodo.{x}' if pd.notna(x) else pd.NA)
      .str.lower()
)

df.loc[is_zenodo, ['url', 'doi_guess']].head()

,url,doi_guess
92,https://zenodo.org/records/10838488,10.5281/zenodo.10838488
96,https://zenodo.org/records/14784563,10.5281/zenodo.14784563
182,https://zenodo.org/records/15047648,10.5281/zenodo.15047648
183,https://zenodo.org/records/15047486,10.5281/zenodo.15047486
184,https://zenodo.org/records/15047671,10.5281/zenodo.15047671


In [72]:
test = '10.5281/zenodo.10838488'
url = f'https://api.datacite.org/dois/{test}'
headers = {'Accept': 'application/vnd.api+json'}
response = requests.get(url, headers=headers)

print(response.text)

{"data":{"id":"10.5281/zenodo.10838488","type":"dois","attributes":{"doi":"10.5281/zenodo.10838488","prefix":"10.5281","suffix":"zenodo.10838488","identifiers":[{"identifier":"oai:zenodo.org:10838488","identifierType":"oai"}],"alternateIdentifiers":[{"alternateIdentifierType":"oai","alternateIdentifier":"oai:zenodo.org:10838488"}],"creators":[{"name":"Catalyst Cooperative","nameType":"Personal","familyName":"Catalyst Cooperative","affiliation":["Catalyst Cooperative"],"nameIdentifiers":[]}],"titles":[{"title":"PUDL Raw EIA Annual Energy Outlook (AEO)"}],"publisher":"Zenodo","container":{},"publicationYear":2024,"subjects":[{"subject":"MW"},{"subject":"MWh"},{"subject":"annual energy outlook"},{"subject":"distribution"},{"subject":"eia"},{"subject":"eia aeo"},{"subject":"electric"},{"subject":"electricity"},{"subject":"energy"},{"subject":"energy consumption"},{"subject":"energy information administration"},{"subject":"energy supply"},{"subject":"federal"},{"subject":"fuel projections"}

In [73]:
df["doi"] = df["doi"].fillna(df["doi_guess"])


getting metadata from datacite

In [91]:
dois = df["doi"].dropna().astype(str).str.strip().str.lower().unique()
dois

array(['10.3886/e226824v1', '10.3886/e223443v1', '10.7910/dvn/f3a62w',
       ..., '10.3886/e238627v1', '10.3886/e220761v1', '10.3886/e237626v1'],
      dtype=object)

In [92]:
unique_dois = (
    df["doi"]
    .dropna()
    .astype(str)
    .str.strip()
    .str.lower()
    .unique()
)

len(unique_dois)

1400

also doi with no version

In [29]:
# grab project id
df["project_id"] = df["url"].astype(str).str.extract(r"/project/(\d+)", expand=False)

# grab version if present, otherwise default to V1 for datalumos project links
df["version"] = df["url"].astype(str).str.extract(r"/version/(V\d+)", expand=False)

mask_datalumos = df["domain"].isin(["www.datalumos.org", "datalumos.org"]) & df["project_id"].notna()
df.loc[mask_datalumos & df["version"].isna(), "version"] = "V1"

# build DOI guess for datalumos rows where we have project_id + version
mask_guess = mask_datalumos & df["version"].notna()
df.loc[mask_guess, "doi_guess"] = (
    "10.3886/E" + df.loc[mask_guess, "project_id"] + df.loc[mask_guess, "version"]
)

df["doi"] = df["doi"].fillna(df["doi_guess"])

df.loc[mask_datalumos, ["url", "project_id", "version", "doi_guess", "doi"]]

,url,project_id,version,doi_guess,doi
1,https://www.datalumos.org/datalumos/project/223443/version/V1/view,223443,V1,10.3886/E223443V1,10.3886/E223443V1
3,https://www.datalumos.org/datalumos/project/222881/version/V1/view,222881,V1,10.3886/E222881V1,10.3886/E222881V1
4,https://www.datalumos.org/datalumos/project/222043/version/V1/view,222043,V1,10.3886/E222043V1,10.3886/E222043V1
5,https://www.datalumos.org/datalumos/project/222044/version/V2/view,222044,V2,10.3886/E222044V2,10.3886/E222044V2
6,https://www.datalumos.org/datalumos/project/222044/version/V2/view,222044,V2,10.3886/E222044V2,10.3886/E222044V2
...,...,...,...,...,...
2015,https://www.datalumos.org/datalumos/project/227006/version/V1/view,227006,V1,10.3886/E227006V1,10.3886/E227006V1
2020,https://www.datalumos.org/datalumos/project/238627/view,238627,V1,10.3886/E238627V1,10.3886/E238627V1
2021,https://www.datalumos.org/datalumos/project/223063/version/V1/view,223063,V1,10.3886/E223063V1,10.3886/E223063V1
2022,https://www.datalumos.org/datalumos/project/220761/view,220761,V1,10.3886/E220761V1,10.3886/E220761V1


lets see some duplicates

In [93]:
dup_rows = df[df["doi"].duplicated(keep=False)]
dup_rows[["doi", "url", "domain", 'title']].head(20)

,doi,url,domain,title
4,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2016 AmeriCorps MES AmeriCorps Member Exit Survey
5,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings National Rates of All Measures by Demographics from the Current Population Survey Civic Engagement and Volunteering Supplement
6,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings National Rates of All Measures from the Current Population Survey Civic Engagement and Volunteering Supplement
7,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017-2023 CEV Findings State-Level Rates of All Measures from the Current Population Survey Civic Engagement and Volunteering Supplement
8,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2017 AmeriCorps MES AmeriCorps Member Exit Survey
10,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2017 CEV Data Current Population Survey Civic Engagement and Volunteering Supplement
11,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2018 AmeriCorps MES AmeriCorps Member Exit Survey
15,10.3886/E222043V1,https://www.datalumos.org/datalumos/project/222043/version/V1/view,www.datalumos.org,2019 AmeriCorps MES AmeriCorps Member Exit Survey
16,10.3886/E224361V1,https://www.datalumos.org/datalumos/project/224361/version/V1/view,www.datalumos.org,2019 CDFI Program Awardee Performance Data Snapshot
17,10.3886/E222044V2,https://www.datalumos.org/datalumos/project/222044/version/V2/view,www.datalumos.org,2019 CEV Data Current Population Survey Civic Engagement and Volunteering Supplement


In [94]:
df['doi'].value_counts()

doi
10.3886/E222581V1    51
10.3886/E223141V1    48
10.3886/E229201V1    37
10.3886/E223001V1    30
10.3886/E224621V1    25
                     ..
10.3886/E240277V1     1
10.3886/E239260V1     1
10.3886/E239259V1     1
10.3886/E239108V1     1
10.3886/E237626V1     1
Name: count, Length: 1401, dtype: int64

In [95]:
df[df['doi'] == '10.3886/E222581V1']

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess
1142,_datasets/medical-expenditure-panel-survey-meps-topics.md,data_rescue_project,Medical Expenditure Panel Survey (MEPS) Topics,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp,None,2025-03-18,False,None,Health & Healthcare,1,Medical Expenditure Panel Survey (MEPS) Topics,447,https://www.datalumos.org/datalumos/project/222581/version/V1/view,"TSV, ZIP",Finished,0.243,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1156,_datasets/meps-topics-access-to-health-care.md,data_rescue_project,MEPS Topics Access to Health Care,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=1Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Access to Health Care,448,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1157,_datasets/meps-topics-childrens-health.md,data_rescue_project,MEPS Topics Children's Health,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=2Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Children's Health,449,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1158,_datasets/meps-topics-childrens-insurance-coverage.md,data_rescue_project,MEPS Topics Children's Insurance Coverage,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=3Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Children's Insurance Coverage,450,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1159,_datasets/meps-topics-dental-visitsuseevents-and-expenditures.md,data_rescue_project,MEPS Topics Dental Visits/Use/Events and Expenditures,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=47Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Dental Visits/Use/Events and Expenditures,451,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1160,_datasets/meps-topics-disability.md,data_rescue_project,MEPS Topics Disability,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=20Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Disability,452,https://www.datalumos.org/datalumos/project/222581/version/V1/view,PDF,Finished,0.000,2025-03-03,"DRP, DL",None,www.datalumos.org,10.3886/E222581V1,222581,V1,10.3886/E222581V1
1161,_datasets/meps-topics-doctor-visitsuseevents-and-expenditures.md,data_rescue_project,MEPS Topics Doctor Visits/Use/Events and Expenditures,Agency for Healthcare Research and Quality,Department of Health and Human Services,meps.ahrq.gov,https://meps.ahrq.gov/mepsweb/data_stats/MEPS_topics.jsp?topicid=21Z-1,None,2025-03-18,False,None,Health & Healthcare,1,MEPS Topics Doctor Visits/Use/Events and Expenditures,453,https://www.datalumos.org/datalumos/project/

metadata

In [96]:
MAILTO = "maja.murawka@student.uva.nl"
session = requests.Session()
session.headers.update({
    "Accept": "application/vnd.api+json",
    "User-Agent": f"metadata-harvest/1.0 (mailto:{MAILTO})"
})


rows = []
for doi in unique_dois:
    url = f"https://api.datacite.org/dois/{doi}"
    try:
        r = session.get(url, timeout=30)

        if r.status_code == 429:
            time.sleep(2)
            r = session.get(url, timeout=30)

        if r.status_code != 200:
            rows.append({"doi": doi, "dc_ok": False, "dc_status": r.status_code, "dc_error": r.text[:200]})
            continue

        data = r.json()
        attrs = data.get("data", {}).get("attributes", {})
        rel = data.get("data", {}).get("relationships", {})

        row = {
            "doi": doi,
            "dc_ok": True,
            "dc_status": 200,
            "dc_raw": data,
        }

        for k in [
            "prefix", "suffix", "publisher", "publicationYear", "language",
            "version", "url", "contentUrl", "schemaVersion", "source",
            "isActive", "state", "reason", "metadataVersion",
            "created", "registered", "published", "updated",
            "viewCount", "downloadCount", "referenceCount", "citationCount",
            "partCount", "partOfCount", "versionCount", "versionOfCount"
        ]:
            row[f"dc_{k}"] = attrs.get(k)

        types = attrs.get("types") or {}
        row["dc_types_ris"] = types.get("ris")
        row["dc_types_bibtex"] = types.get("bibtex")
        row["dc_types_citeproc"] = types.get("citeproc")
        row["dc_types_schemaOrg"] = types.get("schemaOrg")
        row["dc_types_resourceTypeGeneral"] = types.get("resourceTypeGeneral")

        row["dc_identifiers"] = attrs.get("identifiers")
        row["dc_alternateIdentifiers"] = attrs.get("alternateIdentifiers")

        titles = attrs.get("titles") or []
        row["dc_title"] = titles[0].get("title") if titles else None
        row["dc_titles_all"] = titles

        creators = attrs.get("creators") or []
        row["dc_creators_all"] = creators
        row["dc_creators_names"] = "; ".join([c.get("name","") for c in creators if c.get("name")]) or None
        row["dc_creators_types"] = "; ".join([c.get("nameType","") for c in creators if c.get("nameType")]) or None

        # --- contributors ---
        contributors = attrs.get("contributors") or []
        row["dc_contributors_all"] = contributors
        row["dc_contributors_names"] = "; ".join([c.get("name","") for c in contributors if c.get("name")]) or None

        # --- subjects ---
        subjects = attrs.get("subjects") or []
        row["dc_subjects_all"] = subjects
        row["dc_subjects"] = "; ".join([s.get("subject","") for s in subjects if s.get("subject")]) or None

        # --- dates ---
        dates = attrs.get("dates") or []
        row["dc_dates_all"] = dates
        # handy “picked” fields
        collected = [d.get("date") for d in dates if (d.get("dateType") or "").lower() == "collected"]
        issued = [d.get("date") for d in dates if (d.get("dateType") or "").lower() == "issued"]
        row["dc_date_collected"] = collected[0] if collected else None
        row["dc_date_issued"] = issued[0] if issued else None

        # --- descriptions ---
        descs = attrs.get("descriptions") or []
        row["dc_descriptions_all"] = descs
        abstract = [d.get("description") for d in descs if (d.get("descriptionType") or "").lower() == "abstract"]
        row["dc_abstract"] = abstract[0] if abstract else None

        # --- geoLocations ---
        geos = attrs.get("geoLocations") or []
        row["dc_geoLocations_all"] = geos
        row["dc_geo_places"] = "; ".join(
            [g.get("geoLocationPlace","") for g in geos if g.get("geoLocationPlace")]
        ) or None

        # --- rights & funding ---
        row["dc_rightsList"] = attrs.get("rightsList")
        row["dc_fundingReferences"] = attrs.get("fundingReferences")

        # --- related identifiers ---
        relids = attrs.get("relatedIdentifiers") or []
        row["dc_relatedIdentifiers_all"] = relids
        row["dc_isVersionOf"] = "; ".join(
            [x.get("relatedIdentifier","") for x in relids if (x.get("relationType") or "") == "IsVersionOf"]
        ) or None

        # --- relationships (structured links) ---
        # e.g. provider/client/versionOf ids
        def rel_id(name):
            d = rel.get(name, {}).get("data")
            if isinstance(d, dict):
                return d.get("id")
            if isinstance(d, list):
                return "; ".join([x.get("id","") for x in d if isinstance(x, dict) and x.get("id")]) or None
            return None

        row["dc_rel_client"] = rel_id("client")
        row["dc_rel_provider"] = rel_id("provider")
        row["dc_rel_media"] = rel_id("media")
        row["dc_rel_versionOf"] = rel_id("versionOf")
        row["dc_rel_versions"] = rel_id("versions")
        row["dc_rel_parts"] = rel_id("parts")
        row["dc_rel_partOf"] = rel_id("partOf")
        row["dc_rel_citations"] = rel_id("citations")
        row["dc_rel_references"] = rel_id("references")

        rows.append(row)

    except Exception as e:
        rows.append({"doi": doi, "dc_ok": False, "dc_status": None, "dc_error": str(e)})

datacite_df = pd.DataFrame(rows)
datacite_df.head()

doi  dc_ok  dc_status  \
0   10.3886/e226824v1   True        200   
1   10.3886/e223443v1   True        200   
2  10.7910/dvn/f3a62w   True        200   
3   10.3886/e222881v1   True        200   
4   10.3886/e222043v1   True        200   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [97]:
df["doi_norm"] = (
    df["doi"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace({"nan": None, "none": None, "": None})
)

datacite_df["doi_norm"] = (
    datacite_df["doi"]
    .astype(str)
    .str.strip()
    .str.lower()
)

df = df.merge(
    datacite_df.drop(columns=["doi"], errors="ignore"),
    on="doi_norm",
    how="left"
)

In [101]:
df = df.drop(columns='dc_raw')

In [102]:
df.head(5)

file  \
0                                    _datasets/10j-injunctions.md   
1                            _datasets/2002-2024-mbda-grantees.md   
2                           _datasets/2006-iur-public-database.md   
3                       _datasets/2014-minority-veteran-report.md   
4  _datasets/2016-americorps-mes-americorps-member-exit-survey.md   

                schema                                              title  \
0  data_rescue_project                                  10(j) Injunctions   
1  data_rescue_project                            2002-2024 MBDA Grantees   
2  data_rescue_project                           2006 IUR Public Database   
3  data_rescue_project                       2014 Minority Veteran Report   
4  data_rescue_project  2016 AmeriCorps MES AmeriCorps Member Exit Survey   

                                                                organization  \
0                                             National Labor Relations Board   
1                                       Minority Business Development Agency   
2                                            Environmental Protection Agency   
3  Office of Information and Technology - IT Operations and Services (ITOPS)   
4                                                                 AmeriCorps   

                            agency             websites  \
0   National Labor Relations Board             nlrb.gov   
1           Department of Commerce             mbda.gov   
2  Environmental Protection Agency              epa.gov   
3   Department of Veterans Affairs          data.va.gov   
4                       AmeriCorps  data.americorps.gov   

                                                                                                           data_source  \
0                                                  https://www.nlrb.gov/what-we-do/investigate-charges/10j-injunctions   
1                                                                     https://www.mbda.gov/research/data/mbda-grantees   
2                                    https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database   
3                                                                          https://www.data.va.gov/stories/s/gavm-n6bm   
4  https://data.americorps.gov/National-Service/2016-AmeriCorps-MES-AmeriCorps-Member-Exit-Survey/wqhv-fm5d/about_data   

  description last_modified metadata_available  \
0        None    2025-04-20               True   
1        None    2025-04-02              False   
2        None    2025-09-18               True   
3        None    2025-03-17              False   
4        None    2025-03-10              False   

                                                                        metadata_url  \
0                                                                               None   
1                                                                               None   
2  https://www.epa.gov/chemical-data-reporting/downloadable-2006-iur-public-database   
3                                                                               None   
4                                                                               None   

                                                      category  \
0                                           Labor & Employment   
1                                           Business & Economy   
2                                        Climate & Environment   
3              Science & Research; Military & Veterans Affairs   
4  Humanitarian & Disaster Relief; Military & Veterans Affairs   

   resource_count                                      project_title    id  \
0               1                                  10(j) Injunctions   759   
1               1                            2002-2024 MBDA Grantees   696   
2               1                           2006 IUR Public Database  1280   
3               1                       2014 Minority Veteran Report   412 

In [103]:
no_doi = df[df["doi"].isna()]

no_doi

,file,schema,title,organization,agency,websites,data_source,description,last_modified,metadata_available,metadata_url,category,resource_count,project_title,id,url,format,status,size,download_date,maintainer,notes,domain,doi,project_id,version,doi_guess,doi_norm,dc_ok,dc_status,dc_prefix,dc_suffix,dc_publisher,dc_publicationYear,dc_language,dc_version,dc_url,dc_contentUrl,dc_schemaVersion,dc_source,dc_isActive,dc_state,dc_reason,dc_metadataVersion,dc_created,dc_registered,dc_published,dc_updated,dc_viewCount,dc_downloadCount,dc_referenceCount,dc_citationCount,dc_partCount,dc_partOfCount,dc_versionCount,dc_versionOfCount,dc_types_ris,dc_types_bibtex,dc_types_citeproc,dc_types_schemaOrg,dc_types_resourceTypeGeneral,dc_identifiers,dc_alternateIdentifiers,dc_title,dc_titles_all,dc_creators_all,dc_creators_names,dc_creators_types,dc_contributors_all,dc_contributors_names,dc_subjects_all,dc_subjects,dc_dates_all,dc_date_collected,dc_date_issued,dc_descriptions_all,dc_abstract,dc_geoLocations_all,dc_geo_places,dc_rightsList,dc_fundingReferences,dc_relatedIdentifiers_all,dc_isVersionOf,dc_rel_client,dc_rel_provider,dc_rel_media,dc_rel_versionOf,dc_rel_versions,dc_rel_parts,dc_rel_partOf,dc_rel_citations,dc_rel_references,dc_error
44,_datasets/administrative-law-judge-decisions.md,data_rescue_project,Administrative Law Judge Decisions,National Labor Relations Board,National Labor Relations Board,nlrb.gov,https://www.nlrb.gov/cases-decisions/decisions/administrative-law-judge-decisions,None,2025-05-14,False,None,Labor & Employment,1,Administrative Law Judge Decisions,1008,https://nlrbresearch.com/NLRB/NLRB_DB?_search=type%3A+%22ALJ%22,PDF,Finished,0.000,None,NLRB-R,Captured as part of NLRB Research a free database a researcher made.,nlrbresearch.com,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61,_datasets/affirmatively-furthering-fair-housing-affh-data.md,data_rescue_project,Affirmatively Furthering Fair Housing (AFFH) Data,Department of Housing and Urban Development,Department of Housing and Urban Development,hud.gov,https://www.hud.gov/AFFH,None,2025-05-19,True,https://datacatalog.urban.org/dataset/us-department-housing-and-urban-development-affirmatively-furthering-fair-housing-hud-affh,Housing & Community Development,1,Affirmatively Furthering Fair Housing (AFFH) Data,1037,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/04/03/housing-and-communities/hud-affh/data.zip,ZIP,Finished,0.876,2024-12-18,UI,"This dataset contains all data, documentation, and file resources linked to on the main US Department of Housing and Urban Development’s Affirmatively Furthering Fair Housing (AFFH) page and powering the AFFH tool.",urban-data-catalog.s3.us-east-1.amazonaws.com,None,NaN,NaN,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,_datasets/aging-independence-and-disability-agid-program-portal-data.md,data_rescue_project,"AGing, Independence, and Disability (AGID) Program Portal Data",Administration for Community Living,Department of Health and Human Services,agid.acl.gov,https://agid.acl.gov/release.html,None,2025-05-19,True,https://datacatalog.urban.org/dataset/aging-independence-and-disability-agid-program-portal-data,Social Services; Health & Healthcare,1,"AGing, Independence, and Disability (AGID) Program Portal Data",1041,https://urban-data-catalog.s3.us-east-1.amazonaws.com/drupal-root-live/2025/03/28/race-and-equity/agid/data.zip,ZIP,Finished,6.000,2025-03-28,UI,"This resource contains all data from AGing, Independence, and Disability’s Program Data Portal 

In [104]:
df_2 = df[df["doi"].notna()].copy()

In [105]:
df_2.isna().mean().sort_values(ascending=False).head(50)

dc_rel_partOf                   1.000000
dc_rel_parts                    1.000000
dc_reason                       1.000000
dc_contentUrl                   1.000000
dc_rel_references               0.998252
dc_rel_versions                 0.998252
description                     0.996503
dc_rel_citations                0.970862
dc_contributors_names           0.970862
metadata_url                    0.970862
dc_error                        0.951632
notes                           0.603730
dc_geo_places                   0.511072
dc_date_collected               0.480186
dc_subjects                     0.430653
format                          0.303030
size                            0.168998
dc_language                     0.093240
dc_isVersionOf                  0.086830
dc_rel_versionOf                0.085664
project_id                      0.075175
version                         0.075175
dc_version                      0.062937
dc_source                       0.053030
dc_schemaVersion

In [106]:
threshold = 0.97

cols_to_drop = df_2.columns[
    df_2.isna().mean() >= threshold
]

df_2 = df_2.drop(columns=cols_to_drop)

print(f"Dropped {len(cols_to_drop)} columns")

Dropped 10 columns


In [107]:
df_2.columns

Index(['file', 'schema', 'title', 'organization', 'agency', 'websites',
       'data_source', 'last_modified', 'metadata_available', 'category',
       'resource_count', 'project_title', 'id', 'url', 'format', 'status',
       'size', 'download_date', 'maintainer', 'notes', 'domain', 'doi',
       'project_id', 'version', 'doi_guess', 'doi_norm', 'dc_ok', 'dc_status',
       'dc_prefix', 'dc_suffix', 'dc_publisher', 'dc_publicationYear',
       'dc_language', 'dc_version', 'dc_url', 'dc_schemaVersion', 'dc_source',
       'dc_isActive', 'dc_state', 'dc_metadataVersion', 'dc_created',
       'dc_registered', 'dc_published', 'dc_updated', 'dc_viewCount',
       'dc_downloadCount', 'dc_referenceCount', 'dc_citationCount',
       'dc_partCount', 'dc_partOfCount', 'dc_versionCount',
       'dc_versionOfCount', 'dc_types_ris', 'dc_types_bibtex',
       'dc_types_citeproc', 'dc_types_schemaOrg',
       'dc_types_resourceTypeGeneral', 'dc_identifiers',
       'dc_alternateIdentifiers', 'dc_tit

In [108]:
df_2.to_csv('drp_withdoi.csv')

In [109]:
df_2.shape

(1716, 83)